# PyTorch deployment tools and libraries

## Common Example Model

In [1]:
# The following code intantiates the model for use

from torchvision.models import vgg16

model = vgg16(pretrained=True)

/Users/woojin/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/woojin/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# let's print out the number of trainable parameters.
import numpy as np

model_parameters = filter(lambda p: p.requires_grad, model.parameters())

params = sum([np.prod(p.size()) for p in model_parameters])

print(params)

138357544


## Python API

In [ ]:
import system 
import torch

if __name__ == "__main__":
    model = MyModel()
    model.load_state_dict(torch.load(PATH))
    model.eval()
    outputs = model(inputs)
    print(outputs)

## TorchScript

In [3]:
# we can use tracing to convert your model to TorchScript

import torch

model = vgg16(pretrained=True)
example_input = torch.rand(1, 3, 224, 224)
torchscript_model = torch.jit.trace(model, example_input)
torchscript_model.save('./models/traced_vgg16_model.pt')

/Users/woojin/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/woojin/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/woojin/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/jit/_trace.py:1084: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1000 / 1000 (100.0%)
Greatest absolute difference: 2.85996484

In [5]:
# if our model used control flow, we would need to use the annotation method to convert it to TorchScript.

import torch.nn as nn 

class ControlFlowModel(nn.Module):
    def __init__(self, N):
        super(ControlFlowModel, self).__init__()
        self.fc = nn.Linear(N, 100)
        
    def forward(self, input):
        if input.sum() > 0:
            outout = input
        else:
            output = -input
        
        return outout

model = ControlFlowModel(10)
torchscript_model = torch.jit.script(model)
torchscript_model.save('./models/scripted_vgg16_model.pt')

RuntimeError: 

outout is not defined in the false branch:
  File "/var/folders/zr/_c9pchvx7m9274g3530s1jcw0000gn/T/ipykernel_84714/4052292003.py", line 11
    def forward(self, input):
        if input.sum() > 0:
        ~~~~~~~~~~~~~~~~~~~
            outout = input
            ~~~~~~~~~~~~~~
        else:
        ~~~~~
            output = -input
            ~~~~~~~~~~~~~~~ <--- HERE
        
        return outout
and was used here:
  File "/var/folders/zr/_c9pchvx7m9274g3530s1jcw0000gn/T/ipykernel_84714/4052292003.py", line 16
            output = -input
        
        return outout
               ~~~~~~ <--- HERE


# Deploying to a Flask App